In [19]:
# Load run data for left and right of torque: avg and std
import pandas as pd

folder_path = "User_data/User1/day_1/"
torque_run_data_file_l = f"{folder_path}Subj04_run_63_idTrqAve_l.txt"
torque_run_data_file_r = f"{folder_path}Subj04_run_63_idTrqAve_r.txt"
torque_run_data_file_l_std = f"{folder_path}Subj04_run_63_idTrqStd_l.txt"
torque_run_data_file_r_std = f"{folder_path}Subj04_run_63_idTrqStd_r.txt"

# These are all tab seperated use pandas to load them in
# The first column is the gaitPhase not the index
torque_run_data_l = pd.read_csv(torque_run_data_file_l, sep='\t', index_col=0)
torque_run_data_r = pd.read_csv(torque_run_data_file_r, sep='\t', index_col=0)
torque_run_data_l_std = pd.read_csv(torque_run_data_file_l_std, sep='\t', index_col=0)
torque_run_data_r_std = pd.read_csv(torque_run_data_file_r_std, sep='\t', index_col=0)

In [20]:
torque_run_data_l

,pelvis_tilt_moment,pelvis_list_moment,pelvis_rotation_moment,pelvis_tx_force,pelvis_ty_force,pelvis_tz_force,hip_flexion_r_moment,hip_adduction_r_moment,hip_rotation_r_moment,hip_flexion_l_moment,...,lumbar_rotation_moment,knee_angle_r_moment,knee_angle_l_moment,ankle_angle_r_moment,ankle_angle_l_moment,subtalar_angle_r_moment,subtalar_angle_l_moment,mtp_angle_r_moment,mtp_angle_l_moment,Unnamed: 24
gaitPhase,,,,,,,,,,,,,,,,,,,,,
1.0,-6.98,-61.56,33.23,140.05,-6.58,333.93,39.15,-12.32,-5.34,-49.40,...,-19.25,-1.35,-32.21,-7.18,-2.61,-2.93,0.59,0.06,-0.04,NaN
2.0,-45.72,33.55,-26.87,-2.12,148.67,-212.55,28.86,8.63,-2.47,-57.99,...,-19.98,0.38,-29.84,-5.38,-2.81,-2.44,1.43,0.06,-0.07,NaN
3.0,-41.56,3.29,-15.50,-3.20,-44.05,-81.14,31.12,3.76,-2.89,-82.87,...,-20.22,0.88,-36.33,-4.14,-3.27,-1.95,3.16,0.06,-0.09,NaN
4.0,-46.74,-26.49,-0.09,-10.28,-198.48,53.09,33.00,-1.23,-3.20,-105.02,...,-18.61,1.10,-40.64,-3.12,-1.03,-1.58,4.50,0.07,0.02,NaN
5.0,-77.51,4.66,-15.24,-84.28,-113.67,-134.69,29.25,6.85,-1.84,-108.57,...,-15.33,2.09,-35.23,-2.12,4.05,-1.36,5.13,0.08,0.28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.0,-3.48,-35.95,1.67,17.36,-8.71,127.97,21.20,-10.52,-5.79,-41.40,...,-18.29,-10.83,-29.21,-20.54,-0.36,-6.47,-0.20,0.04,0.03,NaN
97.0,-9.29,-34.10,-5.51,49.59,38.43,65.70,24.67,-5.68,-5.09,-43.26,...,-19.76,-7.65,-30.29,-16.19,-0.67,-5.33,0.13,0.04,0.02,NaN
98.0,-16.47,-19.02,-17.65,35.01,96.99,-44.07,25.00,0.70,-4.39,-45.13,...,-20.93,-5.37,-30.44,-12.57,-0.77,-4.40,0.36,0.05,0.03,NaN


In [444]:
from sqlalchemy import create_engine, Column, Integer, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.types import String, Date

# Create a new SQLite database (or connect to an existing one)
DATABASE_URL = 'sqlite:///example.db'
engine = create_engine(DATABASE_URL)

# Create a base class
class CustomBase:
    @classmethod
    def get_or_create(cls, session, model, **kwargs):
        """
        Tries to get an instance of the model using the provided filters.
        If no such instance exists, it creates a new one.
        :param session: SQLAlchemy session.
        :param model: SQLAlchemy model class.
        :param kwargs: Fields used for querying and creating the object.
        :return: Tuple (object, created), where `object` is the instance and
                 `created` is True if the instance was created, False otherwise.
        """
        # Prepare a dictionary to hold any foreign key transformations

        new_kwargs = {}

        # Convert object-based foreign keys to IDs, e.g., user -> user_id
        for key, value in kwargs.items():
            if hasattr(value, 'id'):  # Check if value is a model instance
                new_kwargs[key + '_id'] = value.id  # Convert to foreign key field
            else:
                new_kwargs[key] = value  # Keep the original field



        # Query using the transformed dictionary
        instance = session.query(model).filter_by(**new_kwargs).first()

        if instance:
            return instance, False
        else:
            instance = model(**new_kwargs)
            session.add(instance)
            session.commit()
            return instance, True



# Define a new Base class that inherits from both CustomBase and the SQLAlchemy declarative base
Base = declarative_base(cls=CustomBase)


# Define Soleus and SoleusSide with one-to-one relationships
class SoleusRightSide(Base):
    __tablename__ = 'soleus_right_side'

    id = Column(Integer, primary_key=True)
    soleus_id = Column(Integer, ForeignKey('soleus.id'), unique=True)
    soleus = relationship('Soleus', back_populates='right')
    force_avg = Column(Integer)
    force_std = Column(Integer)


class SoleusLeftSide(Base):
    __tablename__ = 'soleus_left_side'

    id = Column(Integer, primary_key=True)
    soleus_id = Column(Integer, ForeignKey('soleus.id'), unique=True)
    soleus = relationship('Soleus', back_populates='left')
    force_avg = Column(Integer)
    force_std = Column(Integer)

class Soleus(Base):
    __tablename__ = 'soleus'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='soleus')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('SoleusRightSide', back_populates='soleus', uselist=False, foreign_keys=[SoleusRightSide.soleus_id])
    left = relationship('SoleusLeftSide', back_populates='soleus', uselist=False, foreign_keys=[SoleusLeftSide.soleus_id])


class TibialisAnteriorRightSide(Base):
    __tablename__ = 'tibialis_anterior_right_side'

    id = Column(Integer, primary_key=True)
    tibialis_anterior_id = Column(Integer, ForeignKey('tibialis_anterior.id'), unique=True)
    tibialis_anterior = relationship('TibialisAnterior', back_populates='right')
    force_avg = Column(Integer)
    force_std = Column(Integer)

class TibialisAnteriorLeftSide(Base):
    __tablename__ = 'tibialis_anterior_left_side'

    id = Column(Integer, primary_key=True)
    tibialis_anterior_id = Column(Integer, ForeignKey('tibialis_anterior.id'), unique=True)
    tibialis_anterior = relationship('TibialisAnterior', back_populates='left')
    force_avg = Column(Integer)
    force_std = Column(Integer)

class TibialisAnterior(Base):
    __tablename__ = 'tibialis_anterior'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='tibialis_anterior')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('TibialisAnteriorRightSide', back_populates='tibialis_anterior', uselist=False, foreign_keys=[TibialisAnteriorRightSide.tibialis_anterior_id])
    left = relationship('TibialisAnteriorLeftSide', back_populates='tibialis_anterior', uselist=False, foreign_keys=[TibialisAnteriorLeftSide.tibialis_anterior_id])

# Define Medial Gastrocnemius and MedialGastrocnemiusSide with one-to-one relationships
class MedialGastrocnemiusRightSide(Base):
    __tablename__ = 'medial_gastrocnemius_right_side'

    id = Column(Integer, primary_key=True)
    medial_gastrocnemius_id = Column(Integer, ForeignKey('medial_gastrocnemius.id'), unique=True)
    medial_gastrocnemius = relationship('MedialGastrocnemius', back_populates='right')
    force_avg = Column(Integer)
    force_std = Column(Integer)

class MedialGastrocnemiusLeftSide(Base):
    __tablename__ = 'medial_gastrocnemius_left_side'

    id = Column(Integer, primary_key=True)
    medial_gastrocnemius_id = Column(Integer, ForeignKey('medial_gastrocnemius.id'), unique=True)
    medial_gastrocnemius = relationship('MedialGastrocnemius', back_populates='left')
    force_avg = Column(Integer)
    force_std = Column(Integer)

class MedialGastrocnemius(Base):
    __tablename__ = 'medial_gastrocnemius'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='medial_gastrocnemius')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('MedialGastrocnemiusRightSide', back_populates='medial_gastrocnemius', uselist=False, foreign_keys=[MedialGastrocnemiusRightSide.medial_gastrocnemius_id])
    left = relationship('MedialGastrocnemiusLeftSide', back_populates='medial_gastrocnemius', uselist=False, foreign_keys=[MedialGastrocnemiusLeftSide.medial_gastrocnemius_id])

class LateralGastrocnemiusRightSide(Base):
    __tablename__ = 'lateral_gastrocnemius_right_side'

    id = Column(Integer, primary_key=True)
    lateral_gastrocnemius_id = Column(Integer, ForeignKey('lateral_gastrocnemius.id'), unique=True)
    lateral_gastrocnemius = relationship('LateralGastrocnemius', back_populates='right')
    force_avg = Column(Integer)
    force_std = Column(Integer)


class LateralGastrocnemiusLeftSide(Base):
    __tablename__ = 'lateral_gastrocnemius_left_side'

    id = Column(Integer, primary_key=True)
    lateral_gastrocnemius_id = Column(Integer, ForeignKey('lateral_gastrocnemius.id'), unique=True)
    lateral_gastrocnemius = relationship('LateralGastrocnemius', back_populates='left')
    force_avg = Column(Integer)
    force_std = Column(Integer)


class LateralGastrocnemius(Base):
    __tablename__ = 'lateral_gastrocnemius'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='lateral_gastrocnemius')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('LateralGastrocnemiusRightSide', back_populates='lateral_gastrocnemius', uselist=False, foreign_keys=[LateralGastrocnemiusRightSide.lateral_gastrocnemius_id])
    left = relationship('LateralGastrocnemiusLeftSide', back_populates='lateral_gastrocnemius', uselist=False, foreign_keys=[LateralGastrocnemiusLeftSide.lateral_gastrocnemius_id])


# Define Hip and HipSide with one-to-one relationships
class HipRightSide(Base):
    __tablename__ = 'hip_right_side'

    id = Column(Integer, primary_key=True)
    hip_id = Column(Integer, ForeignKey('hips.id'), unique=True)
    hip = relationship('Hip', back_populates='right')
    flexion_avg = Column(Integer)
    adduction_avg = Column(Integer)
    rotation_avg = Column(Integer)
    flexion_std = Column(Integer)
    adduction_std = Column(Integer)
    rotation_std = Column(Integer)

class HipLeftSide(Base):
    __tablename__ = 'hip_left_side'

    id = Column(Integer, primary_key=True)
    hip_id = Column(Integer, ForeignKey('hips.id'), unique=True)
    hip = relationship('Hip', back_populates='left')
    flexion_avg = Column(Integer)
    adduction_avg = Column(Integer)
    rotation_avg = Column(Integer)
    flexion_std = Column(Integer)
    adduction_std = Column(Integer)
    rotation_std = Column(Integer)

class Hip(Base):
    __tablename__ = 'hips'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='hip')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('HipRightSide', back_populates='hip', uselist=False, foreign_keys=[HipRightSide.hip_id])
    left = relationship('HipLeftSide', back_populates='hip', uselist=False, foreign_keys=[HipLeftSide.hip_id])

# Define other parts (Ankle, Knee, Pelvis) similarly
class AnkleLeftSide(Base):
    __tablename__ = 'ankle_left_sides'

    id = Column(Integer, primary_key=True)
    ankle_id = Column(Integer, ForeignKey('ankles.id'), unique=True)
    ankle = relationship('Ankle', back_populates='left')
    subtalar_angle_avg = Column(Integer)
    angle_avg = Column(Integer)
    subtalar_angle_std = Column(Integer)
    angle_std = Column(Integer)

class AnkleRightSide(Base):
    __tablename__ = 'ankle_right_sides'

    id = Column(Integer, primary_key=True)
    ankle_id = Column(Integer, ForeignKey('ankles.id'), unique=True)
    ankle = relationship('Ankle', back_populates='right')
    subtalar_angle_avg = Column(Integer)
    angle_avg = Column(Integer)
    subtalar_angle_std = Column(Integer)
    angle_std = Column(Integer)


class Ankle(Base):
    __tablename__ = 'ankles'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='ankle')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('AnkleRightSide', back_populates='ankle', uselist=False, foreign_keys=[AnkleRightSide.ankle_id])
    left = relationship('AnkleLeftSide', back_populates='ankle', uselist=False, foreign_keys=[AnkleLeftSide.ankle_id])

class KneeRightSide(Base):
    __tablename__ = 'knee_right_side'

    id = Column(Integer, primary_key=True)
    knee_id = Column(Integer, ForeignKey('knees.id'), unique=True)
    knee = relationship('Knee', back_populates='right')
    angle_avg = Column(Integer)
    angle_std = Column(Integer)

class KneeLeftSide(Base):
    __tablename__ = 'knee_left_side'

    id = Column(Integer, primary_key=True)
    knee_id = Column(Integer, ForeignKey('knees.id'), unique=True)
    knee = relationship('Knee', back_populates='left')
    angle_avg = Column(Integer)
    angle_std = Column(Integer)

class Knee(Base):
    __tablename__ = 'knees'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='knee')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('KneeRightSide', back_populates='knee', uselist=False, foreign_keys=[KneeRightSide.knee_id])
    left = relationship('KneeLeftSide', back_populates='knee', uselist=False, foreign_keys=[KneeLeftSide.knee_id])

class PelvisRightSide(Base):
    __tablename__ = 'pelvis_right_side'

    id = Column(Integer, primary_key=True)
    pelvis_id = Column(Integer, ForeignKey('pelvis.id'), unique=True)
    pelvis = relationship('Pelvis', back_populates='right')
    tilt_angle_avg = Column(Integer)
    list_angle_avg = Column(Integer)
    rotation_angle_avg = Column(Integer)
    tilt_angle_std = Column(Integer)
    list_angle_std = Column(Integer)
    rotation_angle_std = Column(Integer)

class PelvisLeftSide(Base):
    __tablename__ = 'pelvis_left_side'

    id = Column(Integer, primary_key=True)
    pelvis_id = Column(Integer, ForeignKey('pelvis.id'), unique=True)
    pelvis = relationship('Pelvis', back_populates='left')
    tilt_angle_avg = Column(Integer)
    list_angle_avg = Column(Integer)
    rotation_angle_avg = Column(Integer)
    tilt_angle_std = Column(Integer)
    list_angle_std = Column(Integer)
    rotation_angle_std = Column(Integer)

class Pelvis(Base):
    __tablename__ = 'pelvis'

    id = Column(Integer, primary_key=True)
    gait_phase = relationship('GaitPhase', back_populates='pelvis')
    gait_phase_id = Column(Integer, ForeignKey('gait_phases.id'))
    right = relationship('PelvisRightSide', back_populates='pelvis', uselist=False, foreign_keys=[PelvisRightSide.pelvis_id])
    left = relationship('PelvisLeftSide', back_populates='pelvis', uselist=False, foreign_keys=[PelvisLeftSide.pelvis_id])

class GaitPhase(Base):
    __tablename__ = 'gait_phases'

    id = Column(Integer, primary_key=True)
    exercise_unit_id = Column(Integer, ForeignKey('exercise_units.id'))
    exercise_unit = relationship('ExerciseUnit', back_populates='gait_phases')
    phase = Column(Integer)
    pelvis = relationship('Pelvis', back_populates='gait_phase')
    ankle = relationship('Ankle', back_populates='gait_phase')
    knee = relationship('Knee', back_populates='gait_phase')
    hip = relationship('Hip', back_populates='gait_phase')

    soleus = relationship('Soleus', back_populates='gait_phase')
    medial_gastrocnemius = relationship('MedialGastrocnemius', back_populates='gait_phase')
    lateral_gastrocnemius = relationship('LateralGastrocnemius', back_populates='gait_phase')
    tibialis_anterior = relationship('TibialisAnterior', back_populates='gait_phase')


class ExerciseUnit(Base):
    __tablename__ = 'exercise_units'

    id = Column(Integer, primary_key=True)
    run_id = Column(Integer, ForeignKey('runs.id'))
    unit_count = Column(Integer)
    run = relationship('Run', back_populates='kilometers')
    walk_id = Column(Integer, ForeignKey('walks.id'))
    walk = relationship('Walk', back_populates='kilometers')
    jump_id = Column(Integer, ForeignKey('jumps.id'))
    jump = relationship('Jump', back_populates='thirty_reps')
    squat_id = Column(Integer, ForeignKey('squats.id'))
    squat = relationship('Squat', back_populates='thirty_reps')
    land_id = Column(Integer, ForeignKey('lands.id'))
    land = relationship('Land', back_populates='thirty_reps')
    lunge_id = Column(Integer, ForeignKey('lunges.id'))  # Add this line
    lunge = relationship('Lunge', back_populates='thirty_reps')  # Add this line
    gait_phases = relationship('GaitPhase', back_populates='exercise_unit')


class Run(Base):
    __tablename__ = 'runs'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(Date)
    duration = Column(Integer)
    speed= Column(Integer)
    user = relationship('User', back_populates='runs')
    kilometers = relationship('ExerciseUnit', back_populates='run')

class Walk(Base):
    __tablename__ = 'walks'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(Date)
    duration = Column(Integer)
    speed = Column(Integer)
    user = relationship('User', back_populates='walks')
    kilometers = relationship('ExerciseUnit', back_populates='walk')


class Jump(Base):
    __tablename__ = 'jumps'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(Date)
    duration = Column(Integer)
    user = relationship('User', back_populates='jumps')
    thirty_reps = relationship('ExerciseUnit', back_populates='jump')



class Squat(Base):
    __tablename__ = 'squats'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(Date)
    duration = Column(Integer)
    user = relationship('User', back_populates='squats')
    thirty_reps = relationship('ExerciseUnit', back_populates='squat')


class Land(Base):
    __tablename__ = 'lands'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(Date)
    duration = Column(Integer)
    user = relationship('User', back_populates='lands')
    thirty_reps = relationship('ExerciseUnit', back_populates='land')



class Lunge(Base):
    __tablename__ = 'lunges'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    date = Column(Date)
    duration = Column(Integer)
    user = relationship('User', back_populates='lunges')
    thirty_reps = relationship('ExerciseUnit', back_populates='lunge')



# Example of a simple model (table) definition
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    height = Column(Integer)
    weight = Column(Integer)
    runs = relationship('Run', back_populates='user')
    walks = relationship('Walk', back_populates='user')
    jumps = relationship('Jump', back_populates='user')
    squats = relationship('Squat', back_populates='user')
    lands = relationship('Land', back_populates='user')
    lunges = relationship('Lunge', back_populates='user')


# Create the database tables
Base.metadata.create_all(engine)

# Create a session factory
Session = sessionmaker(bind=engine)
session = Session()


/var/folders/fx/n_73dlgs4x70_fjl7lbqf9600000gp/T/ipykernel_22891/3781477613.py:50: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base(cls=CustomBase)


In [445]:
from datetime import datetime





var_dict = {
    'pelvis_tilt' : 'pelvis',
    'pelvis_list' : 'pelvis',
    'pelvis_rotation' : 'pelvis',
    'hip_flexion_r' : 'hip',
    'hip_flexion_l' : 'hip',
    'hip_adduction_r' : 'hip',
    'hip_adduction_l' : 'hip',
    'hip_rotation_r' : 'hip',
    'hip_rotation_l' : 'hip',
    'ankle_angle_r' : 'ankle',
    'ankle_angle_l' : 'ankle',
    'subtalar_angle_r' : 'ankle',
    'subtalar_angle_l' : 'ankle',
    'knee_angle_r' : 'knee',
    'knee_angle_l' : 'knee',
    'soleus_r' : 'soleus',
    'soleus_l' : 'soleus',
    'tib_ant_r' : 'tibialis_anterior',
    'med_gas_r' : 'medial_gastrocnemius',
    'lat_gas_r' : 'lateral_gastrocnemius',
    }

unfound_vars = set()
std_filled = 0
pelvis_sides_created = 0


def load_data_into_db(file, type_exercise_unit, speed, data_type, mode, side, user, date=datetime.now()):
    data = pd.read_csv(file, sep='\t')
    data = data.drop(data.columns[0], axis=1)
    data.reset_index(drop=True, inplace=True)
    global std_filled
    global pelvis_sides_created



    for index, row in data.iloc[:-1].iterrows():

        # Query based on unique attributes (date, speed, user)
        if type_exercise_unit == 'run':
            exercise, created = CustomBase.get_or_create(session, Run, date=date, speed=speed, user=user)
        elif type_exercise_unit == 'walk':
            exercise, created = CustomBase.get_or_create(session, Walk, date=date, speed=speed, user=user)
        elif type_exercise_unit == 'jump':
            exercise, created = CustomBase.get_or_create(session, Jump, date=date, user=user)
        elif type_exercise_unit == 'squat':
            exercise, created = CustomBase.get_or_create(session, Squat, date=date, user=user)
        elif type_exercise_unit == 'land':
            exercise, created = CustomBase.get_or_create(session, Land, date=date, user=user)
        elif type_exercise_unit == 'lunge':
            exercise, created = CustomBase.get_or_create(session, Lunge, date=date, user=user)

        # Ensure the corresponding `ExerciseUnit` is tied to the correct exercise type
        exercise_unit, created = CustomBase.get_or_create(
            session, ExerciseUnit, run=exercise if type_exercise_unit == 'run' else None,
            walk=exercise if type_exercise_unit == 'walk' else None,
            jump=exercise if type_exercise_unit == 'jump' else None,
            squat=exercise if type_exercise_unit == 'squat' else None,
            land=exercise if type_exercise_unit == 'land' else None,
            lunge=exercise if type_exercise_unit == 'lunge' else None
        )

        # Continue with the rest of your logic
        gait_phase, created = CustomBase.get_or_create(session, GaitPhase, exercise_unit=exercise_unit, phase=index)
        
        if data_type == 'ikAng':
            pelvis, created = CustomBase.get_or_create(session, Pelvis, gait_phase=gait_phase)
            ankle, created = CustomBase.get_or_create(session, Ankle, gait_phase=gait_phase)
            knee, created = CustomBase.get_or_create(session, Knee, gait_phase=gait_phase)
            hip, created = CustomBase.get_or_create(session, Hip, gait_phase=gait_phase)
        if data_type == 'musFor':
            soleus, created = CustomBase.get_or_create(session, Soleus, gait_phase=gait_phase)
            medial_gastrocnemius, created = CustomBase.get_or_create(session, MedialGastrocnemius, gait_phase=gait_phase)
            lateral_gastrocnemius, created = CustomBase.get_or_create(session, LateralGastrocnemius, gait_phase=gait_phase)
            tibialis_anterior, created = CustomBase.get_or_create(session, TibialisAnterior, gait_phase=gait_phase)


        session.flush()
        session.commit()

        if side == 'l':
            if data_type == 'ikAng':
                pelvis_side = pelvis.left
                if not pelvis_side:
                    pelvis_side = PelvisLeftSide(pelvis=pelvis)
                    session.add(pelvis_side)
                    pelvis_sides_created += 1

                ankle_side = ankle.left
                if not ankle_side:
                    ankle_side = AnkleLeftSide(ankle=ankle)
                    session.add(ankle_side)

                knee_side = knee.left
                if not knee_side:
                    knee_side = KneeLeftSide(knee=knee)
                    session.add(knee_side)

                hip_side = hip.left
                if not hip_side:
                    hip_side = HipLeftSide(hip=hip)
                    session.add(hip_side)
            
            if data_type == 'musFor':

                soleus_side = soleus.left
                if not soleus_side:
                    soleus_side = SoleusLeftSide(soleus=soleus)
                    session.add(soleus_side)

                medial_gastrocnemius_side = medial_gastrocnemius.left
                if not medial_gastrocnemius_side:
                    medial_gastrocnemius_side = MedialGastrocnemiusLeftSide(medial_gastrocnemius=medial_gastrocnemius)
                    session.add(medial_gastrocnemius_side)

                lateral_gastrocnemius_side = lateral_gastrocnemius.left
                if not lateral_gastrocnemius_side:
                    lateral_gastrocnemius_side = LateralGastrocnemiusLeftSide(lateral_gastrocnemius=lateral_gastrocnemius)
                    session.add(lateral_gastrocnemius_side)

                tibialis_anterior_side = tibialis_anterior.left
                if not tibialis_anterior_side:
                    tibialis_anterior_side = TibialisAnteriorLeftSide(tibialis_anterior=tibialis_anterior)
                    session.add(tibialis_anterior_side)

        elif side == 'r':
            if data_type == 'ikAng':
                pelvis_side = pelvis.right
                if not pelvis_side:
                    pelvis_side = PelvisRightSide(pelvis=pelvis)
                    session.add(pelvis_side)
                    pelvis_sides_created += 1

                ankle_side = ankle.right
                if not ankle_side:
                    ankle_side = AnkleRightSide(ankle=ankle)
                    session.add(ankle_side)

                knee_side = knee.right
                if not knee_side:
                    knee_side = KneeRightSide(knee=knee)
                    session.add(knee_side)

                hip_side = hip.right
                if not hip_side:
                    hip_side = HipRightSide(hip=hip)
                    session.add(hip_side)
            
            if data_type == 'musFor':

                soleus_side = soleus.right
                if not soleus_side:
                    soleus_side = SoleusRightSide(soleus=soleus)
                    session.add(soleus_side)

                medial_gastrocnemius_side = medial_gastrocnemius.right
                if not medial_gastrocnemius_side:
                    medial_gastrocnemius_side = MedialGastrocnemiusRightSide(medial_gastrocnemius=medial_gastrocnemius)
                    session.add(medial_gastrocnemius_side)

                lateral_gastrocnemius_side = lateral_gastrocnemius.right
                if not lateral_gastrocnemius_side:
                    lateral_gastrocnemius_side = LateralGastrocnemiusRightSide(lateral_gastrocnemius=lateral_gastrocnemius)
                    session.add(lateral_gastrocnemius_side)

                tibialis_anterior_side = tibialis_anterior.right
                if not tibialis_anterior_side:
                    tibialis_anterior_side = TibialisAnteriorRightSide(tibialis_anterior=tibialis_anterior)
                    session.add(tibialis_anterior_side)
        
        session.flush()
        session.commit()  


        for variable in data.columns:
            if variable not in var_dict:
                unfound_vars.add(variable)
            if variable in var_dict:
                
                #set angle data
                if data_type == 'ikAng':
                    if var_dict[variable] == 'pelvis':
                        if mode == 'Ave':
                            db_var = variable.replace('pelvis_', '') + '_angle_avg'
                            setattr(pelvis_side, db_var, row[variable])
                        elif mode == 'Std':
                            db_var = variable.replace('pelvis_', '') + '_angle_std'
                            setattr(pelvis_side, db_var, row[variable])
                            print(vars(pelvis_side))
                            print(pelvis_side.tilt_angle_std)
                            std_filled += 1
                    elif var_dict[variable] == 'hip':
                        if mode == 'Ave':
                            db_var = variable.replace('hip_', '').replace('_r', '') + '_angle_avg'
                            setattr(hip_side, db_var, row[variable])
                        elif mode == 'Std':
                            db_var = variable.replace('hip_', '').replace('_r','') + '_angle_std'
                            setattr(hip_side, db_var, row[variable])
                    elif var_dict[variable] == 'ankle':
                        if mode == 'Ave':
                            db_var = variable.replace('ankle_', '').replace('_r', '') + '_avg'
                            setattr(ankle_side, db_var, row[variable])
                        elif mode == 'Std':
                            db_var = variable.replace('ankle_', '').replace('_r', '') + '_std'
                            setattr(ankle_side, db_var, row[variable])
                    elif var_dict[variable] == 'knee':
                        if mode == 'Ave':
                            db_var = variable.replace('knee_', '').replace('_r', '') + '_avg'
                            setattr(knee_side, db_var, row[variable])
                        elif mode == 'Std':
                            db_var = variable.replace('knee_', '').replace('_r', '') + '_std'
                            setattr(knee_side, db_var, row[variable])
                    
                    session.flush()
                    session.commit()

                    
                if data_type == 'musFor':
                    #set muscle force data    
                    if var_dict[variable] == 'soleus':
                        if mode == 'Ave':
                            setattr(soleus_side, 'force_avg', row[variable])
                        elif mode == 'Std':
                            setattr(soleus_side, 'force_std', row[variable])
                    elif var_dict[variable] == 'tibialis_anterior':
                        if mode == 'Ave':
                            setattr(tibialis_anterior_side, 'force_avg', row[variable])
                        elif mode == 'Std':
                            setattr(tibialis_anterior_side, 'force_std', row[variable])
                    elif var_dict[variable] == 'medial_gastrocnemius':
                        if mode == 'Ave':
                            setattr(medial_gastrocnemius_side, 'force_avg', row[variable])
                        elif mode == 'Std':
                            setattr(medial_gastrocnemius_side, 'force_std', row[variable])
                    elif var_dict[variable] == 'lateral_gastrocnemius':
                        if mode == 'Ave':
                            setattr(lateral_gastrocnemius_side, 'force_avg', row[variable])
                        elif mode == 'Std':
                            setattr(lateral_gastrocnemius_side, 'force_std', row[variable])
                
                    session.flush()
                    session.commit()  
                
        

        

def load_torque_data_into_db(file, exercise_unit, speed, mode, side, user, date=datetime.now()):
    pass

def load_muscle_force_data_into_db(file, exercise_unit, speed, mode, side, user, date=datetime.now()):
    pass


In [446]:
user_data = {
    'weight': 75.2,
    'height': 180,
    'name': 'User1'
}

#create a new User object
new_user = User(**user_data)
session.add(new_user)

#date = today minus 30 days
from datetime import datetime, timedelta


import os

#iterate over all day_{} folders inside the User_data/User1 folder
for day_folder in os.listdir("User_data/User1")[2:4]:

    if not os.path.isdir(f"User_data/User1/{day_folder}"):
        continue
    
    date = (datetime.now() - timedelta(days=30 - int(day_folder.split('_')[-1]))).date()

    # for each folder iterate over all files in the folder
    for day_number, file in enumerate(os.listdir(f"User_data/User1/{day_folder}")):
        print('processing file:', file, "of day: ", day_folder)
        #check the file name to determine the type of data
        #split file name pattern by "_"
        file_name_parts = file.split("_")
        #iterate over filename parts to get the data type
        i = 1

        #get exercise unit type and it second part
        exercise_unit = file_name_parts[i]
        i += 1

        #get the speed from the next part if exercise is walk or run
        if exercise_unit in ['walk', 'run']:
            speed = file_name_parts[2]
            i+=1
        else:
            speed = None
        
        #get the type of data (ikAng, idTrq, musFor) and mode (Ave, Std) from this part which is like "ikAngAve"
        data_type = file_name_parts[i].replace('Ave', '').replace('Std', '')
        mode = file_name_parts[i].replace(data_type, '')
        i += 1

        #get the side from the last part
        side = file_name_parts[-1].replace('.txt', '')


        #if it is angle join data (ikAng) run the load angle data function
        if data_type == 'ikAng':
            load_data_into_db(f"User_data/User1/{day_folder}/{file}", exercise_unit, speed, data_type, mode, side, new_user, date=date)  

        #if it is muscle force join data (musFor) run the load muscle force data function
        if data_type == 'musFor':
           load_data_into_db(f"User_data/User1/{day_folder}/{file}", exercise_unit, speed,data_type, mode, side, new_user, date=date)

session.commit()

print('Unfound variables:', unfound_vars)

processing file: Subj04_walk_09_idTrqStd_r.txt of day:  day_5
processing file: Subj04_walk_45_idTrqStd_l.txt of day:  day_5
processing file: Subj04_run_81_ikAngAve_r.txt of day:  day_5
processing file: Subj04_jump_ikAngAve_l.txt of day:  day_5
processing file: Subj04_walk_45_musForStd_r.txt of day:  day_5
processing file: Subj04_squat_musForStd_l.txt of day:  day_5
processing file: Subj04_lunge_ikAngAve_l.txt of day:  day_5
processing file: Subj04_run_63_ikAngAve_r.txt of day:  day_5
processing file: Subj04_walk_36_ikAngStd_r.txt of day:  day_5
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12d5dd250>, 'tilt_angle_std': np.float64(0.82)}
0.82
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12d5dd250>, 'list_angle_std': np.float64(0.57)}
0.82
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12d5dd250>, 'rotation_angle_std': np.float64(0.86)}
0.82
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12

In [447]:
std_filled
#pelvis_sides_created

7722

In [448]:
#inspect the data in the database for user 1 on day 1
user = session.query(User).filter_by(name='User1').first()
print(user.name, user.height, user.weight)

date = datetime.now()

jump = session.query(Jump).filter_by().all()
print(len(jump))
jump = jump[0]
print(vars(jump.thirty_reps[0]))

gait_phases = jump.thirty_reps[0].gait_phases 
#print(vars(gait_phases[11]))


print('Number of exercise_units:', len(user.runs), len(user.walks), len(user.jumps), len(user.squats), len(user.lands), len(user.lunges))


all_tibialis_sides = session.query(TibialisAnteriorLeftSide).all() + session.query(TibialisAnteriorRightSide).all()
print('Nubmer of Tibiali_sides:' , len(all_tibialis_sides))

all_ankle_sides = session.query(AnkleLeftSide).all() + session.query(AnkleRightSide).all()
print('Nubmer of ankle_sides:' , len(all_ankle_sides))
for ankle in all_ankle_sides:
    print(vars(ankle))

all_pevis = session.query(Pelvis).all() 
print('Nubmer of pelvis:' , len(all_pevis))

all_pevis_sides = session.query(PelvisLeftSide).all() + session.query(PelvisRightSide).all()
print('Nubmer of pelvis_sides:' , len(all_pevis_sides))

seen = []
avg = 0
std = 0
both = 0
for pevlis in all_pevis:

    if not pevlis.left or not pevlis.right:
        print('Left or right side not found')
        continue
    if pevlis.left.id in seen:
        print('Duplicate found left:', pevlis.left.id)
    seen.append(pevlis.left.id)
    if pevlis.right.id in seen:
        print('Duplicate found:', pevlis.right.id)
    seen.append(pevlis.right.id)
    
    if pevlis.left.list_angle_avg:
        avg += 1
    if pevlis.left.list_angle_std:
        std += 1
    if pevlis.right.list_angle_avg:
        avg += 1
    if pevlis.right.list_angle_std:
        std += 1
    
    if pevlis.left.tilt_angle_avg and pevlis.left.tilt_angle_std:
        both += 1
    if pevlis.right.tilt_angle_avg and pevlis.right.tilt_angle_std:
        both += 1


    if not pevlis.left:
        print('Left side not found')
    if not pevlis.right:
        print('Right side not found')

print('Number of pelvis sides with tilt angle avg:', avg)
print('Number of pelvis sides with tilt angle std:', std)
print('Number of pelvis sides with both tilt angle avg and std:', both)

User1 180 75.2
2
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12c6bf5f0>, 'unit_count': None, 'jump_id': 1, 'lunge_id': None, 'id': 11, 'run_id': None, 'walk_id': None, 'squat_id': None, 'land_id': None}
Number of exercise_units: 3 6 1 1 1 1
Nubmer of Tibiali_sides: 2601
Nubmer of ankle_sides: 2601
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12c83fdd0>, 'angle_avg': None, 'subtalar_angle_avg': None, 'subtalar_angle_std': 2.45, 'ankle_id': 4, 'id': 1, 'angle_std': 0.6}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12c83da90>, 'angle_avg': None, 'subtalar_angle_avg': None, 'subtalar_angle_std': 2.38, 'ankle_id': 6, 'id': 2, 'angle_std': 2.44}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x12c83d370>, 'angle_avg': None, 'subtalar_angle_avg': None, 'subtalar_angle_std': 1.59, 'ankle_id': 8, 'id': 3, 'angle_std': 1.96}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1

In [438]:
pelvis_sid = session.query(PelvisLeftSide).first()

print(vars(pelvis_sid))

pelvis_sid.rotation_angle_std = 2

session.flush()
session.commit()

pelvis_sid = session.query(PelvisLeftSide).first()

print(vars(pelvis_sid))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x163ccae10>, 'tilt_angle_avg': -7.18, 'list_angle_avg': -1.16, 'rotation_angle_avg': -1.58, 'tilt_angle_std': None, 'list_angle_std': None, 'rotation_angle_std': 2, 'pelvis_id': 100, 'id': 1}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x163ccae10>, 'tilt_angle_avg': -7.18, 'list_angle_avg': -1.16, 'rotation_angle_avg': -1.58, 'tilt_angle_std': None, 'list_angle_std': None, 'rotation_angle_std': 2, 'pelvis_id': 100, 'id': 1}
